# Simple example for concurrent processing of coroutines with `asyncio`

In [1]:
import asyncio
import datetime
import time
import contextlib

In [2]:
def now():
    return datetime.datetime.now().time()

## Define a simple context manager for measuring the execution time of statements inside its `with` block

In [3]:
@contextlib.contextmanager
def measure_time():
    start = time.perf_counter()
    yield
    end = time.perf_counter()
    print(f"Needed {end - start} seconds.")

## Create a simple coroutine that waits for some time before returning its result
Note that this coroutine simply sleeps and returns control to the event loop. Instead of `await`ing `asyncio.sleep(...)`, one could `await` any kind of I/O operation:
* HTTP requests and other network activity
* file system operations
* executing database queries
* ...

In [4]:
async def wait(seconds):
    print(f"{now()} begin {seconds}")
    await asyncio.sleep(seconds)
    print(f"{now()} end   {seconds}")
    return seconds

## Multiple sequential `await` statements
Note that the runtime is the sum of the runtimes of the coroutines.

In [5]:
with measure_time():
    print(await wait(0.5))
    print(await wait(0.8))
    print(await wait(0.3))    

11:14:15.358519 begin 0.5
11:14:15.859767 end   0.5
0.5
11:14:15.859944 begin 0.8
11:14:16.661000 end   0.8
0.8
11:14:16.661161 begin 0.3
11:14:16.961748 end   0.3
0.3
Needed 1.6034360009944066 seconds.


## Parallelization with `asyncio.gather`
Two coroutines can be sent to the event loop in parallel, such that the longest running coroutine determines the total runtime.

While one coroutine is `await`ing results, other coroutines can be scheduled by the event loop and do useful work!

In [6]:
with measure_time():
    print(await asyncio.gather(wait(0.5), wait(0.8), wait(0.3)))

11:14:16.974729 begin 0.5
11:14:16.974829 begin 0.8
11:14:16.974869 begin 0.3
11:14:17.275403 end   0.3
11:14:17.475998 end   0.5
11:14:17.775543 end   0.8
[0.5, 0.8, 0.3]
Needed 0.8023969390196726 seconds.
